# tensorflow pistachio


In [5]:
import tensorflow as tf
print(tf.__version__)

2024-05-01 15:16:55.622305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


## arff to csv





In [6]:
import pandas as pd 
from scipy.io import arff
import os 
label_mapping = {'Kirmizi_Pistachio': 0, 'Siit_Pistachio': 1}

def load_arff_file(input_arff: str) -> pd.DataFrame:
    """convert arff file to parquet"""
    if not os.path.exists(input_arff):
        raise ValueError(f"input file '{input_arff}' does not exist")
    print(f'loading arff file {input_arff}')
    data, meta = arff.loadarff(input_arff)
    print(f"arff metadata: {meta}")
    df = pd.DataFrame(data)
    df['Class'] = df['Class'].astype(str).map(label_mapping)
    
    return df
##################

arff_filename = './data/Pistachio_16_Features_Dataset.arff'
csv_filename = './data/pistachio_16.csv'
if not os.path.exists(csv_filename):
    df = load_arff_file(arff_filename)
    df.head()
    df.to_csv(csv_filename, index=False, header=True)
    print(f'wrote file to {csv_filename}')
else:
    print(f'{csv_filename} exists')


./data/pistachio_16.csv exists


## dataset


In [23]:
import numpy as np
from typing import List
# want a stratified split here
def split_csv_data(infilename: str, filenames: List[str], fractions: List[float]):
    df = pd.read_csv(infilename, header=0)
    df = df.sample(frac=1.0, random_state=34)
    columns = df.columns
    print(f'total_records = {len(df)}')
    if len(filenames) != len(fractions):
        raise ValueError('list of filenames must be of same length as split fractions')
    renorm = sum(fractions)
    lower_bound = 0
    df['split_var'] = np.random.uniform(size=len(df))

    for filename, frac in zip(filenames, fractions):
        upper_bound = lower_bound + frac/renorm
        this_data = df.loc[(df.split_var >= lower_bound) & (df.split_var < upper_bound) ][columns]
        this_data.to_csv(filename, index=False, header=True)
        print(f'wrote {len(this_data)} records to {filename}')
        lower_bound = upper_bound
#################################
        
    

    
train_filename = './data/pistachio_train.csv'
valid_filename = './data/pistachio_valid.csv'
test_filename = './data/pistachio_test.csv'
filenames = [train_filename, valid_filename, test_filename]
fractions = [0.75, 0.10, 0.15]

if not (os.path.exists(train_filename) and os.path.exists(test_filename) and os.path.exists(valid_filename)):
    split_csv_data(csv_filename, filenames, fractions )
else:
    print(f'{filenames} exist')






total_records = 2148
wrote 1589 records to ./data/pistachio_train.csv
wrote 221 records to ./data/pistachio_valid.csv
wrote 338 records to ./data/pistachio_test.csv


In [24]:
def map_func(features, labels):
    return tf.transpose(tf.stack([features[k] for k in features])), tf.reshape(labels,[-1,1])

# use dataset.map to concatenate feature dictionary into tensor
pistachio_train_batches = tf.data.experimental.make_csv_dataset(
    train_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)
pistachio_test_data = tf.data.experimental.make_csv_dataset(
    test_filename,
    batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)
validation_data = tf.data.experimental.make_csv_dataset(
    valid_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)

In [10]:
batch = 0
for feature_batch, label_batch in pistachio_train_batches.take(2):
    # print(f'{batch}, {label_batch.shape}')
    # cat_batch = tf.stack([feature_batch['AREA'],feature_batch['PERIMETER']],axis=1)
    # cat_batch = tf.stack([feature_batch[k] for k in feature_batch],axis=1)

    # batch += 1
    
    print("'label': {}".format(label_batch))
    # print(cat_batch)
    print(f"features batch shape: {feature_batch.shape}")
    # print(feature_batch.shape)
    

'label': [[0]
 [0]
 [0]
 [0]]
features batch shape: (4, 16)
'label': [[0]
 [0]
 [1]
 [0]]
features batch shape: (4, 16)


2024-05-01 15:18:13.790145: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Model

In [47]:
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization
# from tensorflow.keras import Model

# class PistachioModel(Model):
#     def __init__(self, units: int=10):
#         super().__init__()
#         self._units = units
    

#     def build(self, input_shape):
#         self.bn = BatchNormalization(axis=0, input_shape=input_shape)
#         self.d1 = Dense(self._units, activation='relu', input_shape=input_shape)
#         self.d2 = Dense(self._units)
#         self.lout = Dense(1, activation='sigmoid')
        


#     def call(self, x):
#         x = self.bn(x)
#         x = self.d1(x)
#         x = self.d2(x)
#         return self.lout(x)

# # Create an instance of the model
# model = PistachioModel()

## Keras model.fit api

In [48]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#               metrics=['accuracy', 'auc'])







In [49]:
# model.fit(pistachio_train_batches, epochs=10)

Epoch 1/10


ValueError: Exception encountered when calling PistachioModel.call().

[1mShapes used to initialize variables must be fully-defined (no `None` dimensions). Received: shape=(None,) for variable path='pistachio_model_10/batch_normalization_9/gamma'[0m

Arguments received by PistachioModel.call():
  • x=tf.Tensor(shape=(None, 16), dtype=float32)

## sequential model

In [27]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(), 
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy', 'auc', 'precision', 'recall'])

In [28]:
model2.fit(pistachio_train_batches, epochs=50, validation_data=validation_data)


Epoch 1/50
    396/Unknown 3s 2ms/step - accuracy: 0.5830 - auc: 0.6674 - loss: 0.6837 - precision: 0.5160 - recall: 0.7356

2024-05-01 15:33:50.589457: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5832 - auc: 0.6676 - loss: 0.6835 - precision: 0.5162 - recall: 0.7354 - val_accuracy: 0.6878 - val_auc: 0.8447 - val_loss: 0.5815 - val_precision: 0.5734 - val_recall: 0.9111
Epoch 2/50
  1/398 ━━━━━━━━━━━━━━━━━━━━ 34s 88ms/step - accuracy: 0.2500 - auc: 0.3333 - loss: 0.9728 - precision: 0.5000 - recall: 0.3333

2024-05-01 15:33:50.993136: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7146 - auc: 0.7858 - loss: 0.5644 - precision: 0.6774 - recall: 0.7115 - val_accuracy: 0.8507 - val_auc: 0.9291 - val_loss: 0.4144 - val_precision: 0.7664 - val_recall: 0.9111
Epoch 3/50


...

2024-05-01 15:34:48.217374: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-01 15:34:48.360451: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7881 - auc: 0.8777 - loss: 0.4329 - precision: 0.7516 - recall: 0.7571 - val_accuracy: 0.8688 - val_auc: 0.9338 - val_loss: 0.3588 - val_precision: 0.8144 - val_recall: 0.8778
Epoch 50/50


2024-05-01 15:34:49.164437: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-01 15:34:49.274597: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7761 - auc: 0.8588 - loss: 0.4663 - precision: 0.7820 - recall: 0.6989 - val_accuracy: 0.8733 - val_auc: 0.9346 - val_loss: 0.3473 - val_precision: 0.8229 - val_recall: 0.8778


2024-05-01 15:34:50.462286: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-01 15:34:50.613352: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [29]:
model2.evaluate(pistachio_test_batches,verbose=2)


85/85 - 0s - 2ms/step - accuracy: 0.8994 - auc: 0.9482 - loss: 0.3240 - precision: 0.8815 - recall: 0.8686


2024-05-01 15:35:56.070808: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.3240205943584442,
 0.8994082808494568,
 0.948178768157959,
 0.8814814686775208,
 0.8686131238937378]

In [30]:
for features, labels in pistachio_test_batches.take(1):
    predictions = model2(features)
    for p,l in zip(predictions, labels):
        print(f'predicted prob: {p}, label: {l}')

predicted prob: [0.40753356], label: [0]
predicted prob: [0.8371399], label: [1]
predicted prob: [0.23193067], label: [0]
predicted prob: [0.26869354], label: [0]


2024-05-01 15:36:54.435603: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## custom training loop stuff

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )